# Project NoteMarkdown AssistantBot

In [4]:
# imports

import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
import google.generativeai as genai
import time

In [2]:
# Load environment variables in a file called .env
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [5]:
# Start Google API

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
MODEL = "gemini-1.5-flash"
genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
# gemini chat 

chat = gemini.start_chat(history=[])

# Transform Gradio history to Gemini format
def transform_history(history):
    new_history = []
    for chat in history:
        new_history.append({"parts": {"text": chat[0]}, "role": "user"})
        new_history.append({"parts": {"text": chat[1]}, "role": "model"})
    return new_history

def response(message, history):
    global chat
    # The history will be the same as in Gradio, the 'Undo' and 'Clear' buttons will work correctly.
    chat.history = transform_history(history)
    response = chat.send_message(message)
    response.resolve()

    # Each character of the answer is displayed
    for i in range(len(response.text)):
        time.sleep(0.01)
        yield response.text[: i+1]

gr.ChatInterface(response,
                 type="messages",
                 textbox=gr.Textbox(placeholder="Question to Gemini")).launch(debug=True)

In [ ]:
# summarizer class
class TextSummarizer:
    def __init__(self, role):
        
        self.roles = {
            "concise": "You are a master summarizer. Condense the following text into clear, concise markdown bullet points, highlighting the key takeaways and actionable items. Prioritize brevity and clarity.",
            "structured": "You are a logical and organized thinker. Structure the following text into a well-formatted markdown document with headings, subheadings, and lists. Identify the main arguments, supporting evidence, and any logical connections between ideas. Use proper hierarchy and indentation.",
            "detail": "You are a meticulous analyst. Transform the following text into detailed markdown notes. Include all relevant information, even seemingly minor details. Use tables, lists, and code blocks where appropriate to present the information clearly and comprehensively.",
            "creative": "You are a creative writer with a flair for language. Convert the following text into engaging and descriptive markdown notes. Use vivid imagery, metaphors, and storytelling techniques to make the notes memorable and interesting. Focus on capturing the essence and spirit of the original text.",
            "technical": "You are a technical expert in software development. Convert the following technical documentation into well-structured markdown. Pay close attention to code examples, API specifications, and technical terminology. Use code blocks, tables, and lists to enhance readability.",
            "historian": "You are a historian specializing in capturing the essence of historical events. Convert the following text into markdown notes that emphasize the timeline, key figures, and significant events. Use timelines, maps (if applicable and you can generate them), and quotations to bring the history to life.",
            "philosopher": "You are a philosopher skilled in abstract thought. Transform the following philosophical text into well-structured markdown notes. Focus on identifying the core arguments, exploring different perspectives, and highlighting any underlying assumptions. Use quotes and examples to illustrate the concepts.",
            "teacher": "You are a teacher explaining a complex concept. Convert the following text into easy-to-understand markdown notes suitable for students. Use clear language, simple examples, and visual aids (if applicable). Break down complex ideas into smaller, digestible chunks.",
            "researcher": "You are a researcher conducting a literature review. Convert the following research papers into structured markdown notes. Focus on summarizing the key findings, methodologies, and contributions of each paper. Use citations and a consistent formatting style.",
            "assistant": "You are a highly organized personal assistant. Convert the following meeting notes into actionable markdown items. Identify tasks, deadlines, assigned individuals, and any relevant context. Use checklists, tables, and bolding to highlight important information.",
        }
        if role not in self.roles:
            raise ValueError(f"Invalid role: {role}. Available roles are: {', '.join(self.roles.keys())}")
        self.model = genai.GenerativeModel(model_name=MODEL,
                                          system_instruction=self.roles[role])
        self.user_prompt = """
I will provide my note to you. 
If the notes knowledge point is incomplete, please provide complementary information. 
If you are not 100% sure of the answer, write「Need to collect the knowledge point」. 
If code is included in the notes, please provide the code in a code block. 
The content you provide needs to be written in markdown: Here are the notes:\n"""
        
        
        self.response = ""
    def summarize(self, text, stream=False):
        messages = self.user_prompt + text
        self.response = self.model.generate_content(messages, stream=stream)
        return self.response.text
    


In [ ]:
# file handler

class Documents:
    def __init__(self,filepath):
        self.path = filepath
        self.text = ""
        with open(self.path, 'r', encoding='utf-8') as f:
            for line in f:
                self.text += line

fileNote = input("Your Notes: ")
doc = Documents(fileNote)
print(doc.text)